In [1]:
%cd ../

/home/qwj/code/HippoRAG


In [2]:
import os
import shutil

def is_new_files(file_name):
    return not (file_name.startswith("openie_") or file_name.startswith('.') or file_name.endswith("named_entity_output.tsv"))

def move_files(subfolder_name):
    # 定义新子文件夹的名称
    new_subfolder = os.path.join('output', subfolder_name)
    
    # 如果新子文件夹不存在，创建它
    if not os.path.exists(new_subfolder):
        os.makedirs(new_subfolder)

    # 获取文件夹下的所有文件（不包含子文件夹）
    for file_name in os.listdir('output'):
        file_path = os.path.join('output', file_name)
        
        if os.path.isfile(file_path):
            # 检查文件名是否以openie_开头或者以named_entity_output.tsv结尾
            if is_new_files(file_name):
                # 移动文件到新子文件夹
                print(f"Moved: {file_name}")
                shutil.move(file_path, os.path.join(new_subfolder, file_name))

def show_new_files():
    for file_name in os.listdir('output'):
        file_path = os.path.join('output', file_name)
    
        if os.path.isfile(file_path):
            # 检查文件名是否以openie_开头或者以named_entity_output.tsv结尾
            if is_new_files(file_name):
                print(file_name)


## Step 1 Create Retriever Corpus 

In [6]:
!python src/create_graph_v2.py \
    --dataset musique \
    --model_name colbertv2 \
    --extraction_model gpt-3.5-turbo-1106 \
    --threshold 0.8 \
    --extraction_type ner

100%|██████████████████████████████████| 11656/11656 [00:01<00:00, 10142.54it/s]
Correct Wiki Format: 0 out of 11656


In [7]:
show_new_files()

query_to_kb.tsv
rel_kb_to_kb.tsv
musique_queries.entity_and_relations.tsv
kb_to_kb.tsv


## compute similar edges

### kb internal similarity

here we have entities and relations

In [2]:
!CUDA_VISIBLE_DEVICES="4" python src/colbertv2_knn.py --filename output/kb_to_kb.tsv



[Jul 05, 04:03:29] #> Creating directory colbert/indexes/nbits_2 


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 2,
    "kmeans_niters": 20,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 400000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 64,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 180,
    "mask_punctuation": true,
    "checkpoint": "exp\/colbert

### query to kb similarity

In [ ]:
!CUDA_VISIBLE_DEVICES="2,3" python src/colbertv2_knn.py --filename output/query_to_kb.tsv

## Create Real Graph

In [3]:
!python src/create_graph_v2.py \
    --dataset musique \
    --model_name colbertv2 \
    --extraction_model gpt-3.5-turbo-1106 \
    --threshold 0.8 \
    --extraction_type ner \
    --create_graph \
    --cosine_sim_edges

100%|███████████████████████████████████| 11656/11656 [00:01<00:00, 9616.10it/s]
Correct Wiki Format: 0 out of 11656
Creating Graph
100%|██████████████████████████████████| 11656/11656 [00:00<00:00, 37802.77it/s]
Loading Vectors
Augmenting Graph from Similarity
100%|████████████████████████████████| 116708/116708 [00:01<00:00, 86466.27it/s]
Saving Graph
                                                         1
0                                                         
Total Phrases                                       327435
Unique Phrases                                      113611
Number of Individual Triples                        109145
Number of Incorrectly Formatted Triples (ChatGP...    1136
Number of Triples w/o NER Entities (ChatGPT Error)    6762
Number of Unique Individual Triples                 107448
Number of Entities                                  218290
Number of Relations                                 239470
Number of Unique Entities                            9

## Index

In [5]:
!CUDA_VISIBLE_DEVICES="4" python3 src/colbertv2_indexing.py \
    --dataset musique \
    --phrase output/musique_facts_and_sim_graph_phrase_dict_ents_only_lower_preprocess_ner.v3.subset.p \
    --corpus data/musique_corpus.json



[Jul 05, 04:16:20] #> Creating directory data/lm_vectors/colbert/musique/corpus/indexes/exp/colbertv2.0 


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 2,
    "kmeans_niters": 20,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 400000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 64,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 180,
    "mask_punctuation"